# Shell scripts for running each tool


## Ground truth

I got a dataset from The International Genonome Sample Resource. This will act as my ground truth because it already has the populations defined. Thus accuracy can be based on correct cluster assignment and number of clusters. There should be 4 populations in this dataset. https://www.internationalgenome.org/data-portal/data-collection/ggvp-grch37

In [ ]:
wget -c --tries=10 ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/gambian_genome_variation_project_GRCh37/illumina_omni2.5M/ALL_GGVP_autocall_illumina2.5M_b37_aligned_20190710.genotypes.vcf.gz &

## Benchmarking time run time, output, standard error, and memory usage

I may do multiple datasets with varying SNP amounts to see how they compare. I will use three datasets, including the truth, just because it is the largest file I have been able to find.

#### Datasets used:
##### ~49,000SNPs
e.vcf: https://www.frontiersin.org/journals/genetics/articles/10.3389/fgene.2023.1092066/full#supplementary-material;
##### ~1.1mil SNPs
https://www.internationalgenome.org/data-portal/search?q=chr22
##### ~2.3mil SNPs
https://www.internationalgenome.org/data-portal/data-collection/ggvp-grch37


### VCF2PCACluster scripts
#### Ground truth

In [ ]:
# puts job in the background and also outputs the log to a specific folder to track progress and how it ran
nohup ~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_truth_4pop.genotypes.vcf.gz -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_truth_4pop > ~/benchmarking_project/final_project/logs/vcf2pcacluster_real_truth_4pop.log 2>&1 &

VCF2PCACluster output a best cluster assignment of 4, which is inline with the population makeup of the data used.

#### Real data

In [ ]:
wget  -c https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz 


In [ ]:
~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_50000.vcf -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_50000
~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_3.vcf.gz -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_3


### ADMIXTURE Scripts

#### Ground Truth

##### Cross validation script for which K value is best and Main script once K is known
I am only testing 1-5 for K ancestral populations because I know my data has a population of 4, but it would be interesting to see if it detects any populations above 4. This is a large dataset so I don't want to run too many Ks.

In [ ]:
# converting vcf to ped file format then to .bed
plink --vcf real_truth_4pop.genotypes.vcf.gz --recode --out real_truth_4pop
plink --file real_truth_4pop --make-bed --out real_truth_4pop

'''Total genotyping rate is 0.986066.
2331907 variants and 407 people pass filters and QC'''

nohup bash -c 'for K in 1 2 3 4 5; \
do ~/benchmarking_project/admixture/admixture/admixture --cv ~/benchmarking_project/final_project/real_truth_4pop.bed $K | tee real_truth_log${K}.out; done' &
grep -h CV real_truth_log*.out # to view CV error to choose which K is best for further analysis

# main script once K is known
~/benchmarking_project/admixture/admixture/admixture ~/benchmarking_project/final_project/real_truth_4pop.bed 4

ADMIXTURE is outputting that K = 3, as it is the lowest CV error but overall the CV error is not much different from other Ks. So, picking K is subjective to the user in this scenario.

#### Real data

In [ ]:
# converting vcf to ped file format
plink --vcf real_50000.vcf --recode --out real_50000 --chr-set 50
plink --file new_real_50000 --make-bed --out real_50000 --chr-set 50

# removing snps with high missingness
plink --bfile real_50000 --geno 0.05 --make-bed --out filtered_real_50000 --chr-set 50

for K in 1 2 3 4 5 6 7; \
    do ./admixture --cv ~/benchmarking_project/final_project/filtered_real_50000.bed $K | tee real_50000log${K}.out; done
grep -h CV real_50000log*.out
'''Writing output files.
CV error (K=1): 0.60157
CV error (K=2): 0.59049
CV error (K=3): 0.57551
CV error (K=4): 0.59876
CV error (K=5): 0.60458
CV error (K=6): 0.62582
CV error (K=7): 0.61644''' # K = 3 is best

# main command now that we know best K
./admixture ~/benchmarking_project/final_project/filtered_real_50000.bed 3

# next dataset
# converting vcf to ped file format then to .bed
plink --vcf real_3.vcf.gz --recode --out real_3 
plink --file real_3 --make-bed --out real_3
# removing snps with high missingness
plink --bfile real_3 --geno 0.05 --make-bed --out filtered_real_3


nohup bash -c 'for K in 1 2 3 4 5 6; \
do ~/benchmarking_project/admixture/admixture/admixture --cv ~/benchmarking_project/final_project/filtered_real_3.ped $K | tee real_3log${K}.out; done' &
grep -h CV real_3log*.out


### PLINK2 Scripts

#### Ground Truth
I did 6 pcas instead of the default 10, just because it is a large dataset and I know the population number is 4.

In [ ]:
nohup plink2 --ped real_truth_4pop.ped --map real_truth_4pop.map --pca 6 approx --out ~/benchmarking_project/final_project/plink2_results/real_truth_4pop_output  > ~/benchmarking_project/final_project/logs/plink2_real_truth_4pop.log 2>&1 &
# only found 3 clusters/population according to the first 2 PCAs

#### Real Data for Benchmarking

~50,000 SNPs, ~1.1mil SNPs

In [ ]:
plink2 --ped real_50000.ped --map real_50000.map --pca 10 --out ~/benchmarking_project/final_project/plink2_results/real_50000_output

In [ ]:
plink2 --ped real_3.ped --map real_3.map --pca 10 --out ~/benchmarking_project/final_project/plink2_results/real_3_output

In [ ]:
plink2 --ped real_truth_4pop.ped --map real_truth_4pop.map --pca 6 approx --out ~/benchmarking_project/final_project/plink2_results/real_truth_4pop_output